In [1]:
%%capture
%pip install - r requirement.txt
# python==3.6.13

In [26]:
import numpy as np
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpMinimize, LpBinary, LpStatus, value, PULP_CBC_CMD
import pulp as pl
import time
import heapq

debug = 1  # debug mode
TIMEOUT = 500 # timeout 
if debug:
    pd.set_option('display.max_columns', 500)  # show all columns

### PARAMETER INPUT

`None` stands for no boundary.

Hereby its just a demo.

In [3]:
n = 200000 # number of containers

NbvCost = True  # True: max nbv; False: min cost

minTotalNbv = 10000000
maxTotalNbv = 350000000

minTotalCost = 1000000
maxTotalCost = 300000000

fleetAgeLowBound = [None, 3, None, None, None]
fleetAgeUpBound = [3, 6, None, None, None]
fleetAgeLimit = [0.2, 0.0, None, None, None]
fleetAgeGeq = [True, True, None, None, None] # True: >=, False: <=

fleetAgeAvgLimit = 5.0
fleetAgeAvgGeq = False

OnHireLimit = 0.7

weightedAgeLowBound = [3, 4, None, None, None]
weightedAgeUpBound = [6, 15, None, None, None]
weightedAgeLimit = [0.0, 0.01, None, None, None]
weightedAgeGeq = [True, True, None, None, None]

weightedAgeAvgLimit = 7.0
weightedAgeAvgGeq = False

productType = ['D4H', 'D20', 'D40', None, None]
productLimit = [0.5, 0.1, 0.0, None, None]
productGeq = [True, True, True, None, None]

lesseeType = ['MSC', 'ESSC', 'ONE', None, None]
lesseeLimit = [0.7, 0.2, 0.8, None, None]
lesseeGeq = [False, False, False, None, None]

topLesseeLimit = [0.65, 0.8, None]
topLesseeGeq = [False, False, None]

contractType = ['LT', 'LP', 'LM', None, None]
contractLimit = [0.6, 0.02, 0.0, None, None]
contractGeq = [True, True, True, None, None]

### READ DATA
TODO: read data

In [4]:
rawData = pd.read_excel(io='./20220630_Overall_Fleet_(FIR)_FCI_combined(with planned)_iflorens_simplified.xlsb', sheet_name='Raw', engine='pyxlsb')
data = rawData.iloc[:n, :55]
print(data.shape)

(200000, 9)


In [5]:
if debug:
    # print(data['Fleet Year Fz'].sum() / n)
    # print(data['Age x CEU'].sum() / n)
    # print(data['Product'].value_counts())
    # print(data['Billing Status Fz'].value_counts())
    # print(data['Contract Cust Id'].value_counts())
    print(data['Contract Lease Type'].value_counts())
    pass

LT    98934
LP    90579
LM     1602
ML      629
FF       38
LF       31
LE       17
Name: Contract Lease Type, dtype: int64


### DATA PREPARATION

TODO: change the column name 

#### Container Age

Assign to `FleetAge{0}.format(i)`

In [6]:
def SelectFleetAge(age, i):
    # check input
    if fleetAgeLowBound[i] is None:
        fleetAgeLowBound[i] = -float('inf')
    if fleetAgeUpBound[i] is None:
        fleetAgeUpBound[i] = float('inf')
    
    return 1 if fleetAgeLowBound[i]<=age<fleetAgeUpBound[i] else 0

for i in range(5):
    if fleetAgeLimit[i] is not None:
        column_name = 'FleetAge{0}'.format(i)
        print(column_name)
        data[column_name] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], i), axis=1)

FleetAge0
FleetAge1


#### Status

Assign new column `Status`

In [7]:
def SelectStatus(status):
    # no need to check input
    return 1 if status=='ON' else 0

data['Status'] = data.apply(lambda x: SelectStatus(x['Billing Status Fz']), axis=1)

#### Weighted Age

Assign new column `WeightedAge{0}.format(i)`

In [8]:
def SelectWeightedAge(age, i):
    # check input
    if weightedAgeLowBound[i] is None:
        weightedAgeLowBound[i] = -float('inf')
    if weightedAgeUpBound[i] is None:
        weightedAgeUpBound[i] = float('inf')
    
    return 1 if weightedAgeLowBound[i]<=age<weightedAgeUpBound[i] else 0

for i in range(5):
    if weightedAgeLimit[i] is not None:
        column_name = 'WeightedAge{0}'.format(i)
        print(column_name)
        data[column_name] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], i), axis=1)

WeightedAge0
WeightedAge1


#### Product

Assign new column `ProductType{0}.format(i)`

In [9]:
def SelectProductType(product, i):
    return 1 if product == productType[i] else 0

for i in range(5):
    if productLimit[i] is not None:
        column_name = 'ProductType{0}'.format(i)
        print(column_name)
        data[column_name] = data.apply(lambda x: SelectProductType(x['Product'], i), axis=1)


ProductType0
ProductType1
ProductType2


#### Lessee

Assign new column `Lessee{0}.format(i)`

In [10]:
def SelectLessee(lessee, i):
    return 1 if lessee == lesseeType[i] else 0

for i in range(5):
    if lesseeLimit[i] is not None:
        column_name = 'Lessee{0}'.format(i)
        print(column_name)
        data[column_name] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], i), axis=1)

Lessee0
Lessee1
Lessee2


In [11]:
# ONE HOT -- all lessee
def OneHotLessee(lessee, name):
    return 1 if lessee==name else 0

for lesseeName in data['Contract Cust Id'].value_counts().index:
    data[lesseeName] = data.apply(lambda x: OneHotLessee(x['Contract Cust Id'], lesseeName), axis=1)

#### Contract Type

Assign new column `ContractType{0}.format(i)`

In [12]:
def SelectContractType(contract, i):
    if contract == contractType[i]:
        contract = 1
    else:
        contract = 0
    return contract

for i in range(5):
    if contractLimit[i] is not None:
        column_name = 'ContractType{0}'.format(i)
        print(column_name)
        data[column_name] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], i), axis=1)

ContractType0
ContractType1
ContractType2


### SAVE DATA

In [13]:
if debug:
    data.to_csv('prepared_data_demo.csv')

In [14]:
if debug:
    data = pd.read_csv('./prepared_data_demo.csv')
data.head(3)

/home/liwuchen/anaconda3/envs/port/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unit Id Fz,Cost,Product,Contract Cust Id,Contract Lease Type,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,FleetAge0,FleetAge1,Status,WeightedAge0,WeightedAge1,ProductType0,ProductType1,ProductType2,Lessee0,Lessee1,Lessee2,MSC,ONE,YANG,ESSC,MSK,COSMR,CMA,CUL,TSLINE,NBOS,HALINE,SITC,SINOKOR,INTRAPTE,HTHK,HYDAI,HAPAG,MATSN,DLIM,ATI,AMTC,CSSC,DONGJIN,CHINAV,SMLINE,VSBLLC,HARBOUR,NAMSUNG,SOGESE,SSCI,OML,SINOCL,CKLINE,DCLP,TLINE,SAMSKIP,RCL,MSCL,SYS,MUTO,XIASC,MTT,ZIM,WH,GOODRICH,EMIRATE,HTC,PANOCEAN,IGLO,BOXJ,WKS,LILM,DPL,ENGLEE,RALA,PDZ,BSLI,EQUIPE,MALC,ELAN,AMLSHP,DALCO,OBLS,QEL,MRTS,GOSS,CENTRANS,BISL,GREVILLE,DACL,MELL,APMVAD,WINSMART,MESSINA,EIMSKIP,PORTAU,KNOF,AEDXB3,MINIUS,WFL,PANCON,HAS,FORMOSA,MBF,EGREN,COSLA,ContractType0,ContractType1,ContractType2
0,0,MSDU2865003,3690.0,D20,MSC,LP,3635.841,ON,0.52,0.52,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,MSDU2865019,3690.0,D20,MSC,LP,3635.841,ON,0.52,0.52,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,2,MSDU2865030,3690.0,D20,MSC,LP,3635.841,ON,0.52,0.52,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


### Model Part

#### Prepare Parameters

convert to numpy

In [15]:
nbv = data['Nbv'].to_numpy()
cost = data['Cost'].to_numpy()
status = data['Status'].to_numpy()

fleetAge = []
weightedAge = []
product = []
lessee = []
contract = []
for i in range(5):
    fleetAge.append(data['FleetAge{0}'.format(i)].to_numpy() if fleetAgeLimit[i] is not None else None)
    weightedAge.append(data['WeightedAge{0}'.format(i)].to_numpy() if weightedAgeLimit[i] is not None else None)
    product.append(data['ProductType{0}'.format(i)].to_numpy() if productLimit[i] is not None else None)
    lessee.append(data['Lessee{0}'.format(i)].to_numpy() if lesseeLimit[i] is not None else None)
    contract.append(data['ContractType{0}'.format(i)].to_numpy() if contractLimit[i] is not None else None)

fleetAgeAvg = data['Fleet Year Fz'].to_numpy()
weightedAgeAvg = data['Age x CEU'].to_numpy()

lesseeOneHot = {lesseeName: data[lesseeName].to_numpy() for lesseeName in data['Contract Cust Id'].value_counts().index}


if debug:
    print('nbv:', nbv.shape)
    print('cost:', cost.shape)
    print('status:', status.shape)

    print('fleetAge:', fleetAge)
    print('fleetAgeAvg:', fleetAgeAvg)
    print('weightedAge:', weightedAge)
    print('weightedAgeAvg:', weightedAgeAvg)
    print('productType:', product)
    print('lessee:', lessee)
    print('contractType:', contract)

nbv: (200000,)
cost: (200000,)
status: (200000,)
fleetAge: [array([1, 1, 1, ..., 1, 1, 1]), array([0, 0, 0, ..., 0, 0, 0]), None, None, None]
fleetAgeAvg: [0.52 0.52 0.52 ... 0.24 0.24 0.24]
weightedAge: [array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), None, None, None]
weightedAgeAvg: [0.52  0.52  0.52  ... 0.408 0.408 0.408]
productType: [array([0, 0, 0, ..., 1, 1, 1]), array([1, 1, 1, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), None, None]
lessee: [array([1, 1, 1, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), None, None]
contractType: [array([0, 0, 0, ..., 0, 0, 0]), array([1, 1, 1, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), None, None]


#### Constraints

TODO: confirm constraints


#### Objective

1. min Cost: `Vars @ cost`

2. max Nbv: `Vars @ nbv`

#### Define Problem

In [27]:
def SortTop(l, n):
    topN = heapq.nlargest(n, l, key=lambda x:x[1])
    print(topN)
    return np.sum(np.stack([lesseeOneHot[topN[i][0]] for i in range(n)]), axis=0)

In [28]:
# variables
var = np.array([LpVariable('container_{0}'.format(i), lowBound=0, cat=LpBinary) for i in range(nbv.shape[0])])
# problem
prob = LpProblem("MyProblem", LpMaximize if NbvCost else LpMinimize)

In [29]:
warmProb = LpProblem("WarmProblem", LpMaximize)
warmProb += lpSum(var * 1)
warmProb.solve(PULP_CBC_CMD(msg = False, timeLimit=1))

1

In [30]:
# objective function 
if NbvCost:
    prob += lpSum(var * nbv)
else:
    prob += lpSum(var * cost)

In [31]:
# constraints
numSelected = lpSum(var) # num of selected containers

# nbv
if maxTotalNbv is not None:
    prob += lpSum(var * nbv) <= maxTotalNbv
if minTotalNbv is not None:
    prob += lpSum(var * nbv) >= minTotalNbv
    
# cost
if maxTotalCost is not None:
    prob += lpSum(var * cost) <= maxTotalCost
if minTotalCost is not None:
    prob += lpSum(var * cost) >= minTotalCost

# status
if OnHireLimit is not None:
    prob += lpSum(var * status) >= OnHireLimit * numSelected
    
# container age
if fleetAgeAvgLimit is not None:
    if fleetAgeAvgGeq:
        prob += lpSum(var * fleetAgeAvg) >= fleetAgeAvgLimit * numSelected
    else:
        prob += lpSum(var * fleetAgeAvg) <= fleetAgeAvgLimit * numSelected
for i in range(5):
    if fleetAgeLimit[i] is not None:
        if fleetAgeGeq[i]:
            prob += lpSum(var * fleetAge[i]) >= fleetAgeLimit[i] * numSelected
        else:
            prob += lpSum(var * fleetAge[i]) <= fleetAgeLimit[i] * numSelected

# weighted age
if weightedAgeAvgLimit is not None:
    if weightedAgeAvgGeq:
        prob += lpSum(var * weightedAgeAvg) >= weightedAgeAvgLimit * numSelected
    else:
        prob += lpSum(var * weightedAgeAvg) <= weightedAgeAvgLimit * numSelected
for i in range(5):
    if weightedAgeLimit[i] is not None:
        if weightedAgeGeq[i]:
            prob += lpSum(var * weightedAge[i]) >= weightedAgeLimit[i] * numSelected
        else:
            prob += lpSum(var * weightedAge[i]) <= weightedAgeLimit[i] * numSelected

# product
for i in range(5):
    if productLimit[i] is not None:
        if productGeq[i]:
            prob += lpSum(var * product[i]) >= productLimit[i] * numSelected
        else:
            prob += lpSum(var * product[i]) <= productLimit[i] * numSelected

# # lessee
# for i in range(5):
#     if lesseeLimit[i] is not None:
#         if lesseeGeq[i]:
#             prob += lpSum(var * lessee[i]) >= lesseeLimit[i] * numSelected
#         else:
#             prob += lpSum(var * lessee[i]) <= lesseeLimit[i] * numSelected


# find top1
if topLesseeLimit[0] is not None:
    if topLesseeGeq[0]:
       prob += lpSum(var * SortTop(list({lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}.items()), 1)) >= topLesseeLimit[0] * numSelected
    else:
        prob += lpSum(var * SortTop(list({lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}.items()), 1)) <= topLesseeLimit[0] * numSelected

# find top2
if topLesseeLimit[1] is not None:
    if topLesseeGeq[1]:
        prob += lpSum(var * SortTop(list({lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}.items()), 2)) >= topLesseeLimit[1] * numSelected
    else:
        prob += lpSum(var * SortTop(list({lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}.items()), 2)) <= topLesseeLimit[1] * numSelected


# contract type
for i in range(5):
    if contractLimit[i] is not None:
        if contractGeq[i]:
            prob += lpSum(var * contract[i]) >= contractLimit[i] * numSelected
        else:
            prob += lpSum(var * contract[i]) <= contractLimit[i] * numSelected

# prob.writeLP('problem.lp')

[('MSC', 159212.0)]
[('MSC', 159212.0), ('ONE', 7067.0)]


In [32]:
solver = PULP_CBC_CMD(msg = True, timeLimit=TIMEOUT)
prob.solve(solver)

1

In [33]:
print("==============================================================")
# print(prob)
print("status:", LpStatus[prob.status])
print("==============================================================")
print("target value: ", value(prob.objective))

status: Optimal
target value:  289275890.6370573


In [23]:
# if solution is found
if debug:
    result = np.array([var[i].varValue for i in range(n)])
    print(int(sum(result)), '/', n, 'containers are selected.')

61837 / 200000 containers are selected.


### Analysis

For debug only

TODO: if infeasible, it may take time much longer than expected

In [24]:
if debug:    
    print('======================================================================')
    print("nbv: {0} between {1} - {2}".format(round(sum(result * nbv), 2), minTotalNbv, maxTotalNbv))
    print("cost: {0} between {1} - {2}".format(round(sum(result * cost), 2), minTotalCost, maxTotalCost))
    print('billing status: {0}, -- {1}'.format(sum(result * status)/sum(result), OnHireLimit))

    print("container age: ")
    print('\t container average age is {0}, -- {1}'.format(round(sum(result * fleetAgeAvg)/sum(result), 2), fleetAgeAvgLimit))
    for i in range(5):
        if fleetAgeLimit[i] is not None:
            print("\t container age from {0} to {1} is {2}, -- {3}:".format(fleetAgeLowBound[i], fleetAgeUpBound[i], round(sum(result * fleetAge[i])/sum(result), 2), fleetAgeLimit[i]))

    print("weighted age: ")
    print('\t weighted average age is {0}, -- {1}'.format(round(sum(result * weightedAgeAvg)/sum(result), 2), weightedAgeAvgLimit))
    for i in range(5):
        if weightedAgeLimit[i] is not None:
            print("\t weighted age from {0} to {1} is {2}, -- {3}:".format(weightedAgeLowBound[i], weightedAgeUpBound[i], round(sum(result * weightedAge[i])/sum(result), 2), weightedAgeLimit[i]))    

    print("product: ")
    for i in range(5):
        if productLimit[i] is not None:
            print("\t product {0} is {1}, -- {2}:".format(productType[i], round(sum(result * product[i])/sum(result), 2), productLimit[i]))    
   
    
    # print("lessee: ")
    # for i in range(5):
    #     if lesseeLimit[i] is not None:
    #         print("\t lessee {0} is {1}, -- {2}:".format(lesseeType[i], round(sum(result * lessee[i])/sum(result), 2), lesseeLimit[i]))    

    print('Top lessee:')
    numLessee = {lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}
    sortedLessee = list(numLessee.items())
    top3Lessee = heapq.nlargest(3, sortedLessee, key=lambda x:x[1])
    print('\t top 1 {0} is {1}, -- {2}'.format(top3Lessee[0][0], top3Lessee[0][1] / sum(result), topLesseeLimit[0]))
    print('\t top 2 {0} {1} is {2}, -- {3}'.format(top3Lessee[0][0], top3Lessee[1][0], (top3Lessee[0][1] + top3Lessee[1][1])/ sum(result), topLesseeLimit[1]))

    
    print("contract type: ")
    for i in range(5):
        if contractLimit[i] is not None:
            print("\t contract type {0} is {1}, -- {2}:".format(contractType[i], round(sum(result * contract[i])/sum(result), 2), contractLimit[i])) 


nbv: 289276159.79 between 10000000 - 350000000
cost: 300000000.0 between 1000000 - 300000000
billing status: 0.8619923675927285, -- 0.7
container age: 
	 container average age is 1.12, -- 5.0
	 container age from -inf to 3 is 0.94, -- 0.2:
	 container age from 3 to 6 is 0.05, -- 0.0:
weighted age: 
	 weighted average age is 1.99, -- 7.0
	 weighted age from 3 to 6 is 0.05, -- 0.0:
	 weighted age from 4 to 15 is 0.04, -- 0.01:
product: 
	 product D4H is 0.64, -- 0.5:
	 product D20 is 0.23, -- 0.1:
	 product D40 is 0.1, -- 0.0:
Top lessee:
	 top 1 MSC is 0.6499999999999972, -- 0.65
	 top 2 MSC YANG is 0.6968003384329295, -- 0.8
contract type: 
	 contract type LT is 0.6, -- 0.6:
	 contract type LP is 0.27, -- 0.02:
	 contract type LM is 0.0, -- 0.0:


### Output

In [25]:
if prob.status == 1 or prob.status == 2:
    data = data[['Unit Id Fz', 'Cost', 'Product', 'Contract Cust Id', 'Contract Lease Type', 'Nbv', 'Billing Status Fz', 'Fleet Year Fz', 'Age x CEU']]
    data.insert(loc=0, column="Selected", value=result)
    data.to_csv('demo_result.csv')